In [0]:
%sql
USE CATALOG atliq_project;

Create gold schema

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold;

Create one big table when dimension data are the latest values

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.obt_monthlysales 
(
  date date,
  product_code string,
  product string,
  division string,
  segment string,
  category string,
  variant string,
  customer_code string,
  customer string,
  market string,
  sub_zone string,
  region string,
  platform string,
  channel string,
  sold_quantity integer
)
USING DELTA
PARTITIONED BY (date);

In [0]:
%sql
INSERT INTO gold.obt_monthlysales
( date, 
  product_code,
  product,
  division,
  segment,
  category,
  variant,
  customer_code,
  customer,
  market,
  sub_zone,
  region,
  platform,
  channel,
  sold_quantity
)
SELECT 
  ms.date, 
  ms.product_code, 
  p.product,
  p.division,
  p.segment,
  p.category,
  p.variant, 
  ms.customer_code, 
  c.customer,
  c.market,
  m.sub_zone,
  m.region,
  c.platform,
  c.channel,
  ms.sold_quantity
FROM silver.monthlysales AS ms
LEFT JOIN silver.customers c 
  ON ms.customer_code = c.customer_code 
LEFT JOIN silver.products p 
  ON ms.product_code = p.product_code 
LEFT JOIN silver.markets m
  ON c.market = m.market
WHERE c.is_current = true
  AND p.is_current = true
  AND m.is_current = true

num_affected_rows,num_inserted_rows
1425706,1425706
